<a href="https://colab.research.google.com/github/anuroop-ghosh/Brazilian-eCommerce/blob/main/AIP_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install geohash2

  Preparing metadata (setup.py) ... done
  Created wheel for geohash2: filename=geohash2-1.1-py3-none-any.whl size=15543 sha256=25faf4f023205bf16996c93b29c1829cc48b31c219b7e2e8f084409942e2482a
  Stored in directory: /root/.cache/pip/wheels/c0/21/8d/fe65503f4f439aef35193e5ec10a14adc945e20ff87eb35895
Successfully built geohash2


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
import geohash2 as geohash
from geopy.geocoders import Nominatim
from math import radians, cos, sin, asin, sqrt
from geopy.distance import geodesic
from sklearn.svm import SVC
svm_algo = SVC(C=1.0, kernel='rbf', gamma=1e-3)
from sklearn.linear_model import LogisticRegression as LogR
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.ensemble import GradientBoostingClassifier as GBDT
from xgboost import XGBClassifier as XGB
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint
from sklearn.model_selection  import train_test_split
from sklearn.tree import DecisionTreeClassifier as tree
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from google.colab import files
upload = files.upload()

In [3]:
customers_dataset = pd.read_csv("olist_customers_dataset.csv")
order_items_dataset = pd.read_csv("olist_order_items_dataset.csv")
order_payments_dataset = pd.read_csv("olist_order_payments_dataset.csv")
order_reviews_dataset = pd.read_csv("olist_order_reviews_dataset.csv")
orders_dataset = pd.read_csv("olist_orders_dataset.csv")
products_dataset = pd.read_csv("olist_products_dataset.csv")
sellers_dataset = pd.read_csv("olist_sellers_dataset.csv")
geolocation_dataset = pd.read_csv("olist_geolocation_dataset.csv")
translation_dataset = pd.read_csv("product_category_name_translation.csv")

In [4]:
print("\n Customers Dataset \n\n", customers_dataset.isnull().sum())
#print("\n Geolocation Dataset \n\n", geolocation_dataset.isnull().sum())
print("\n Order Items Dataset \n\n", order_items_dataset.isnull().sum())
print("\n Order Payments Dataset \n\n", order_payments_dataset.isnull().sum())
print("\n Order Reviews Dataset \n\n", order_reviews_dataset.isnull().sum())
print("\n Orders Dataset \n\n", orders_dataset.isnull().sum())
print("\n Products Dataset \n\n", products_dataset.isnull().sum())
print("\n Sellers Dataset \n\n", sellers_dataset.isnull().sum())


 Customers Dataset 

 customer_id                 0
customer_unique_id          0
customer_zip_code_prefix    0
customer_city               0
customer_state              0
dtype: int64

 Order Items Dataset 

 order_id               0
order_item_id          0
product_id             0
seller_id              0
shipping_limit_date    0
price                  0
freight_value          0
dtype: int64

 Order Payments Dataset 

 order_id                0
payment_sequential      0
payment_type            0
payment_installments    0
payment_value           0
dtype: int64

 Order Reviews Dataset 

 review_id                      0
order_id                       0
review_score                   0
review_comment_title       79404
review_comment_message     52429
review_creation_date           0
review_answer_timestamp        0
dtype: int64

 Orders Dataset 

 order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp 

### Turning into dataframes

In [5]:
customers = pd.DataFrame(customers_dataset)
geolocation = pd.DataFrame(geolocation_dataset)
order_items = pd.DataFrame(order_items_dataset)
order_payments = pd.DataFrame(order_payments_dataset)
order_reviews = pd.DataFrame(order_reviews_dataset)
orders = pd.DataFrame(orders_dataset)
products = pd.DataFrame(products_dataset)
sellers = pd.DataFrame(sellers_dataset)
translation = pd.DataFrame(translation_dataset)

In [ ]:
orders.info()
customers.info()
geolocation.info()
order_items.info()
order_payments.info()
order_reviews.info()
products.info()
sellers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       99441 non-null  object
 1   customer_id                    99441 non-null  object
 2   order_status                   99441 non-null  object
 3   order_purchase_timestamp       99441 non-null  object
 4   order_approved_at              99281 non-null  object
 5   order_delivered_carrier_date   97658 non-null  object
 6   order_delivered_customer_date  96476 non-null  object
 7   order_estimated_delivery_date  99441 non-null  object
dtypes: object(8)
memory usage: 6.1+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null 

The above has completed the data import and overview

---



# Perform data conversion and labeling before merging data

##Calculate and label the volumetric weight of the products

In [6]:
#Calculating the volumetric weight
products['product_volumetric_weight_g'] = (products['product_length_cm'] * products['product_width_cm'] * products['product_height_cm'])/5

#Ranges are Below 500g, 500-1500g, 1500-3000g, 3000-5000g and above 5000g
def volumetric_weight_rating(volumetric_weight):
    if volumetric_weight < 500:
      return 1
    elif volumetric_weight < 1500:
      return 2
    elif volumetric_weight < 3000:
      return 3
    elif volumetric_weight < 5000:
      return 4
    else:
      return 5

#Applying the volumetric weight rating
products['volumetric_weight_rating'] = products['product_volumetric_weight_g'].apply(volumetric_weight_rating)
print(products)

                             product_id              product_category_name  \
0      1e9e8ef04dbcff4541ed26657ea517e5                         perfumaria   
1      3aa071139cb16b67ca9e5dea641aaa2f                              artes   
2      96bd76ec8810374ed1b65e291975717f                      esporte_lazer   
3      cef67bcfe19066a932b7673e239eb23d                              bebes   
4      9dc1a7de274444849c219cff195d0b71              utilidades_domesticas   
...                                 ...                                ...   
32946  a0b7d5a992ccda646f2d34e418fff5a0                   moveis_decoracao   
32947  bf4538d88321d0fd4412a93c974510e6  construcao_ferramentas_iluminacao   
32948  9a7c6041fa9592d9d9ef6cfe62a71f8c                    cama_mesa_banho   
32949  83808703fc0706a22e264b9d75f04a2e             informatica_acessorios   
32950  106392145fca363410d287a815be6de4                    cama_mesa_banho   

       product_name_lenght  product_description_lenght  product

In [7]:
# Calculate product volume
products['product_volume_cm3'] = products['product_length_cm'] * products['product_width_cm'] * products['product_height_cm']

# Define a function to rate
print(products[['product_weight_g','product_volume_cm3','product_description_lenght']].describe())

def weight_rating(weight):
    if weight < 300:
        return 1
    elif weight < 700:
        return 2
    elif weight < 1900:
        return 3
    else:
        return 4

def volume_rating(volume):
    if volume < 2880:
        return 1
    elif volume < 6840:
        return 2
    elif volume < 18480:
        return 3
    else:
        return 4

def descleng_rating(des_lenghth):
    if des_lenghth < 339:
        return 1
    elif des_lenghth < 595:
        return 2
    elif des_lenghth < 972:
        return 3
    else:
        return 4

# Application rating function
products['weight_rating'] = products['product_weight_g'].apply(weight_rating)
products['volume_rating'] = products['product_volume_cm3'].apply(volume_rating)
products['descleng_rating'] = products['product_description_lenght'].apply(volume_rating)

# Inspection: Output processed data
volume_rating_counts = products['volume_rating'].value_counts().sort_index()
weight_rating_counts = products['weight_rating'].value_counts().sort_index()
descleng_rating_counts = products['descleng_rating'].value_counts().sort_index()
print(volume_rating_counts,weight_rating_counts,descleng_rating_counts)


       product_weight_g  product_volume_cm3  product_description_lenght
count      32949.000000        32949.000000                32341.000000
mean        2276.472488        16564.096695                  771.495285
std         4282.038731        27057.041650                  635.115225
min            0.000000          168.000000                    4.000000
25%          300.000000         2880.000000                  339.000000
50%          700.000000         6840.000000                  595.000000
75%         1900.000000        18480.000000                  972.000000
max        40425.000000       296208.000000                 3992.000000
volume_rating
1    8205
2    8265
3    8232
4    8249
Name: count, dtype: int64 weight_rating
1    7211
2    8929
3    8550
4    8261
Name: count, dtype: int64 descleng_rating
1    31782
2      559
4      610
Name: count, dtype: int64


## Translation

In [8]:
#Translate to english
# Create the translation dictionary
Translation_dict = dict(zip(translation['product_category_name'], translation['product_category_name_english']))

# Map the 'product_category_name' to its English equivalent
translation['product_category_english'] = translation['product_category_name'].map(Translation_dict)

# Handle missing translations by falling back to the original 'product_category_name'
translation['product_category_english'] = translation['product_category_english'].fillna(translation['product_category_name'])

# Check the output
print(translation[['product_category_name', 'product_category_english']].head())

    product_category_name product_category_english
0            beleza_saude            health_beauty
1  informatica_acessorios    computers_accessories
2              automotivo                     auto
3         cama_mesa_banho           bed_bath_table
4        moveis_decoracao          furniture_decor


##Calculate the distance between zipcodes

In [9]:
# Merge the orders and customers datasets on the customer ID to get customer zip code prefixes in the orders DataFrame.
orders = pd.merge(orders, customers[['customer_id', 'customer_zip_code_prefix']], on='customer_id', how='left')

# Merge the orders dataset with sellers on the 'order_id' column
# This will bring the 'seller_zip_code_prefix' into the orders dataset
orders = pd.merge(orders, order_items[['order_id', 'seller_id']], on='order_id', how='left')
orders = pd.merge(orders, sellers[['seller_id', 'seller_zip_code_prefix']], on='seller_id', how='left')

# Create a dictionary to efficiently map zip codes to coordinates
# Group by 'geolocation_zip_code_prefix' and aggregate coordinates (e.g., using mean)
geolocation_dict = geolocation.groupby('geolocation_zip_code_prefix')[['geolocation_lat', 'geolocation_lng']].mean().to_dict('index')

def calculate_distance(row):
    customer_zip = row['customer_zip_code_prefix']
    seller_zip = row['seller_zip_code_prefix']

# Handle missing zip codes:
    if pd.isna(customer_zip) or pd.isna(seller_zip):
        return np.nan  # Return NaN if either zip code is missing

    # Access the latitude and longitude values directly
    try:  # Try to access coordinates, if available
        customer_coords = (geolocation_dict[customer_zip]['geolocation_lat'], geolocation_dict[customer_zip]['geolocation_lng'])
        seller_coords = (geolocation_dict[seller_zip]['geolocation_lat'], geolocation_dict[seller_zip]['geolocation_lng'])
        return geodesic(customer_coords, seller_coords).km
    except KeyError:  # Handle KeyError if zip code not in dictionary
        return np.nan  # Return NaN if zip code not found

    return geodesic(customer_coords, seller_coords).km

# Calculate distances for each order
orders['distance_km'] = orders.apply(calculate_distance, axis=1)

##Calculate the duration of each process

In [10]:
orders['order_purchase_timestamp'] = pd.to_datetime(orders['order_purchase_timestamp'])
orders['order_approved_at'] = pd.to_datetime(orders['order_approved_at'])
orders['order_delivered_carrier_date'] = pd.to_datetime(orders['order_delivered_carrier_date'])
orders['order_delivered_customer_date'] = pd.to_datetime(orders['order_delivered_customer_date'])
orders['order_estimated_delivery_date'] = pd.to_datetime(orders['order_estimated_delivery_date'])

order_reviews['review_creation_date'] = pd.to_datetime(order_reviews['review_creation_date'])
order_reviews['review_answer_timestamp'] = pd.to_datetime(order_reviews['review_answer_timestamp'])

# Calculate the duration (in hours)
orders['approval_duration_hours'] = (orders['order_approved_at'] - orders['order_purchase_timestamp']).dt.total_seconds() / 3600

# Order shipping time from merchant
orders['delivery_fromseller'] = (orders['order_delivered_carrier_date'] - orders['order_approved_at']).dt.total_seconds() / 3600

# Delivery time of express delivery
orders['delivery_tocusto'] = (orders['order_delivered_customer_date'] - orders['order_delivered_carrier_date']).dt.total_seconds() / 3600

#Calculate the duration of the review answer
order_reviews['review_toanswer'] = (order_reviews['review_answer_timestamp'] - order_reviews['review_creation_date']).dt.total_seconds() / 3600

# Compare order_delivered_custom_date and order_ estimated d_delivery_date
orders['estimated_delivery_later'] = (orders['order_estimated_delivery_date'] > orders['order_delivered_customer_date']).astype(int) #Mark 1 if arrive in time


##Label textual variables

In [11]:
#Mark the content of the review
order_reviews['title_flag'] = order_reviews['review_comment_title'].notna().astype(int)
order_reviews['message_flag'] = order_reviews['review_comment_message'].notna().astype(int)

#Mark the order status
status_mapping = {
    'created': 1,
    'unavailable': 2,
    'canceled': 3,
    'approved': 4,
    'invoiced': 5,
    'processing': 6,
    'shipped': 7,
    'delivered': 8
}

#Convert text to numbers
orders['order_status_numeric'] = orders['order_status'].map(status_mapping)



payment_mapping = {
    'not_defined': 0,
    'voucher': 1,
    'boleto': 2,
    'credit_card': 3,
    'debit_card': 4
}
#Convert text to numbers
order_payments['payment_type_numeric'] = order_payments['payment_type'].map(payment_mapping)


##Calculate customers' loyalty

In [12]:
customer_loyalty = customers.groupby('customer_unique_id')['customer_id'].count().reset_index(name='Loyal customers')

# Merge customers with the loyalty information
customers_with_loyalty = customers.merge(
    customer_loyalty,
    on='customer_unique_id',
    how='left'
)

#Separate orders with one/mulitple review scores

In [13]:
# Create a dictionary with order_id as the key and corresponding rows as the value
order_reviews_dict = {}

# Traverse the data box and store each row by order_id
for index, row in order_reviews.iterrows():
    order_id = row['order_id']
    if order_id not in order_reviews_dict:
        order_reviews_dict[order_id] = []
    order_reviews_dict[order_id].append(row)

# Create two lists to store classification results
repeated_orders = []
unique_orders = []

# Traverse the dictionary to determine if order_id is duplicated
for order_id, rows in order_reviews_dict.items():
    if len(rows) > 1:
        repeated_orders.extend(rows)
    else:
        unique_orders.extend(rows)

# Convert the results to a DataFrame
repeated_orders_df = pd.DataFrame(repeated_orders)
unique_orders_df = pd.DataFrame(unique_orders)

average_repeated_orders = repeated_orders_df.groupby('order_id')['review_score'].mean().reset_index()

# Generate the final dataset
repeated_orders_df.drop('review_score', axis=1, inplace=True)
repeated_orders_df = repeated_orders_df.merge(average_repeated_orders, on='order_id')



#Comments for clarification & reference
repeat_df = (
    repeated_orders_df
    .merge(orders, on='order_id', how='inner')  # Use modified 'orders' with calculated durations
    .merge(order_payments, on='order_id', how='inner')
    .merge(customers, on='customer_id', how='inner')
    .merge(order_items, on='order_id', how='inner')
    .merge(products, on='product_id', how='inner')
    .merge(customer_loyalty, on='customer_unique_id', how='left')
)

unique_df = (
    unique_orders_df
    .merge(orders, on='order_id', how='inner')  # Use modified 'orders' with calculated durations
    .merge(order_payments, on='order_id', how='inner')
    .merge(customers, on='customer_id', how='inner')
    .merge(order_items, on='order_id', how='inner')
    .merge(products, on='product_id', how='inner')
    .merge(customer_loyalty, on='customer_unique_id', how='left')
)


#Joining datasets




In [14]:
repeat_df.drop(['review_creation_date','review_comment_title', 'review_comment_message', 'review_answer_timestamp', 'order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date', 'order_estimated_delivery_date'], axis=1, inplace=True)
unique_df.drop(['review_creation_date','review_comment_title', 'review_comment_message', 'review_answer_timestamp', 'order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date', 'order_estimated_delivery_date'], axis=1, inplace=True)

final_df = pd.concat([repeat_df,unique_df], ignore_index=True)





#Lasso

In [15]:
nodelivery_df = final_df[final_df['order_status_numeric'].isin([1, 2, 3, 4, 5, 6])]
print([col for col in nodelivery_df.columns if 'seller_id' in col])

nodelivery_df.drop(['review_id','order_id', 'customer_id', 'order_status', 'delivery_fromseller', 'delivery_tocusto', 'payment_type','customer_unique_id','product_id','seller_id_x', 'seller_id_y'], axis=1, inplace=True)
#Seller_id error above (for reference)


nodelivery_df.drop(['order_purchase_timestamp','customer_city','customer_state','shipping_limit_date'], axis=1, inplace=True)


delivery_df = final_df[final_df['order_status_numeric'].isin([7,8])]
delivery_df.drop(['review_id','order_id', 'customer_id', 'order_status', 'payment_type','customer_unique_id','product_id','order_purchase_timestamp','customer_city','customer_state','shipping_limit_date','seller_id_x', 'seller_id_y'], axis=1, inplace=True)


['seller_id_x', 'seller_id_y']


##Orders without delivery time

In [16]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso # Import Lasso class from sklearn.linear_model
from sklearn.model_selection import train_test_split # Import train_test_split
from sklearn.metrics import mean_squared_error # Import mean_squared_error


X1 = nodelivery_df.drop(['review_score'], axis=1)  # All features
y1 = nodelivery_df['review_score']  # Target variable

# Handling missing values
X1.fillna(0, inplace=True)
X1 = pd.get_dummies(X1, drop_first=True)  # Exclusive hot encoding

# Standardized features
scaler1 = StandardScaler()
X1_scaled = scaler1.fit_transform(X1)

X1_train, X1_test, y1_train, y1_test = train_test_split(X1_scaled, y1, test_size=0.2, random_state=42)

In [17]:
from sklearn.linear_model import LassoCV

lasso_cv = LassoCV(cv=5)  #cross validation
lasso_cv.fit(X1_train, y1_train)
print(f'Best alpha: {lasso_cv.alpha_}')

Best alpha: 0.020655944899140383


In [20]:
lasso1 = Lasso(alpha= lasso_cv.alpha_) #using the best alpha value
lasso1.fit(X1_train, y1_train)


y1_pred = lasso1.predict(X1_test)
mse1 = mean_squared_error(y1_test, y1_pred)
print(f'Mean Squared Error: {mse1}')


coefficients = pd.DataFrame(lasso1.coef_, index=X1.columns, columns=['Coefficient'])
print(coefficients[coefficients['Coefficient'] != 0])

Mean Squared Error: 1.1450843020714325
                                                    Coefficient
message_flag                                          -0.179523
customer_zip_code_prefix_x                            -0.042968
seller_zip_code_prefix                                -0.185363
distance_km                                            0.149818
estimated_delivery_later                               0.109821
order_status_numeric                                  -0.003020
payment_type_numeric                                  -0.008679
customer_zip_code_prefix_y                            -0.000005
price                                                 -0.019477
product_weight_g                                      -0.039475
product_length_cm                                     -0.056148
product_height_cm                                      0.027374
product_width_cm                                      -0.015393
weight_rating                                          0.083641
d

##Orders with delivery time

In [ ]:
X2 = delivery_df.drop(['review_score'], axis=1)  # All features
y2 = delivery_df['review_score']  # Target variable

# Handling missing values
X2.fillna(0, inplace=True)
X2 = pd.get_dummies(X2, drop_first=True)  # Exclusive hot encoding

# Standardized features
scaler2 = StandardScaler()
X2_scaled = scaler2.fit_transform(X2)

X2_train, X2_test, y2_train, y2_test = train_test_split(X2_scaled, y2, test_size=0.2, random_state=42)


In [ ]:
from sklearn.linear_model import LassoCV

lasso_cv = LassoCV(cv=5)  #cross validation
lasso_cv.fit(X2_train, y2_train)
print(f'Best alpha: {lasso_cv.alpha_}')

Best alpha: 0.0006832769995231099


In [ ]:
lasso2 = Lasso(alpha=0.015)
lasso2.fit(X2_train, y2_train)


y2_pred = lasso2.predict(X2_test)
mse2 = mean_squared_error(y2_test, y2_pred)
print(f'Mean Squared Error: {mse2}')


coefficients = pd.DataFrame(lasso2.coef_, index=X2.columns, columns=['Coefficient'])
print(coefficients[coefficients['Coefficient'] != 0])

Mean Squared Error: 1.4914671321210733
                                              Coefficient
title_flag                                       0.007229
message_flag                                    -0.321216
delivery_fromseller                             -0.095489
delivery_tocusto                                -0.179459
estimated_delivery_later                         0.318805
order_status_numeric                             0.100124
payment_installments                            -0.000074
payment_value                                   -0.068249
customer_zip_code_prefix                         0.009241
order_item_id                                   -0.141586
price                                            0.054815
weight_rating                                    0.005061
product_category_name_beleza_saude               0.007587
product_category_name_cama_mesa_banho           -0.030395
product_category_name_fashion_calcados           0.000852
product_category_name_ferramentas